In [1]:
import time
import visa

In [4]:
script_startup = """loadandrunscript startupscript()
    -- Turn off beeper
    beeper.enable = beeper.OFF
    --
    display.clear()
    -- Restore Series 2602A defaults on ChA and ChB
    -- Changed 9/15/16 by Jon Frost to recall setup 1 instead
    errorqueue.clear()
    -- Select Channel A and Channel B display formats.
    display.screen = display.SMUA_SMUB
    -- Display voltage measurement and sourcing for ChA and ChB.
    smua.sense = smua.SENSE_REMOTE
    smub.sense = smub.SENSE_REMOTE
    --
    display.smua.measure.func = display.MEASURE_DCVOLTS
    display.smub.measure.func = display.MEASURE_DCVOLTS
    smua.source.func = smua.OUTPUT_DCAMPS
    smub.source.func = smub.OUTPUT_DCAMPS
    -- Set the current compliance for ChA ChB
    smua.source.limiti = 1.0
    smub.source.limiti = 1.0
    -- Set the voltage compliance for ChA ChB
    smua.source.limitv = 12
    smub.source.limitv = 12
    -- Set measurement speed for ChA ChB
    -- NPLC is number of power line cycles
    -- 10 (10/60=0.16 secs) is considered high accuracy
    smua.measure.nplc = 10
    smub.measure.nplc = 10
    -- Select measure V auto range.
    smua.measure.autorangev = smua.AUTORANGE_ON
    smub.measure.autorangev = smub.AUTORANGE_ON
    -- open the channels
    smua.source.output = smua.OUTPUT_HIGH_Z
    smub.source.output = smub.OUTPUT_HIGH_Z
    smua.source.offmode = smua.OUTPUT_HIGH_Z
    smub.source.offmode = smub.OUTPUT_HIGH_Z
    setup.save(1)
    setup.poweron = 1
endscript"""

In [5]:
resource_manager = visa.ResourceManager()
gpib_addr=13
source_meter = resource_manager.open_resource(
    "GPIB0::{}::INSTR".format(gpib_addr)
)
# TODO do not reset? Do something else, clear buffers I think
source_meter.write("reset()")

(9, <StatusCode.success: 0>)

In [126]:
script = """display.screen = display.SMUA_SMUB
display.smu{ch}.measure.func = display.MEASURE_DCVOLTS
smu{ch}.source.func = smu{ch}.OUTPUT_DCAMPS
smu{ch}.source.autorangei = smu{ch}.AUTORANGE_ON
smu{ch}.source.leveli = {current}
smu{ch}.source.limitv = {v_limit}
smu{ch}.source.output = smu{ch}.OUTPUT_ON""".format(ch="a",
                                                    current=0.01,
                                                    v_limit=5)


In [9]:
def run_script(script, scriptname):
    instruction = "loadscript {}()\n{}\nendscript\n{}()".format(
        scriptname, script, scriptname
    )
    return source_meter.write(instruction)

In [127]:
run_script(script,"discharge")

(299, <StatusCode.success: 0>)

In [14]:
def read_iv():
    source_meter.write(
        "current, voltage = smua.measure.iv()"
    )
    current = float(source_meter.ask("print(current)"))
    voltage = float(source_meter.ask("print(voltage)"))

    # The Keithley will report totally out of range numbers like 9.91e+37
    # if asked to e.g. charge to 3.9V when the cell is already at 4.2V
    # It is basically its way of saying the condition cannot be achieved
    # The actual current sent is 0.0 A.
#     if abs(current) > 1.0e10 or abs(current) < 1.0e-8:
#         current = 0.0
#     if abs(voltage) < 5.0e-4:
#         voltage = 0.0
    return current, voltage

In [128]:
read_iv()


(0.0100014, 3.76345)

In [122]:
source_meter.write("reset()")

(9, <StatusCode.success: 0>)

In [88]:
source_meter.write("current, voltage = smua.source.iv()")


(37, <StatusCode.success: 0>)

In [ ]:
float(source_meter.query("print(current)"))